<a href="https://colab.research.google.com/github/BjerringNYK/BDS/blob/main/FirstEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Exploratory Data Analysis (EDA) and Visualization


## FINDEX (Global Financial Index Microdata) and it's business context

The Global Findex database is the most comprehensive dataset on adult financial behaviors worldwide, capturing insights into how individuals save, borrow, make payments, and manage financial risks. Initiated by the World Bank in 2011, the dataset is based on nationally representative surveys of over 150,000 adults across more than 140 economies. The 2021 edition provides updated indicators on the use of both formal and informal financial services.

For this analysis, we will conduct an Exploratory Data Analysis (EDA) to uncover key patterns and trends in the financial behaviors of individuals globally. As this analysis is undertaken by a group of finance students with a strong interest in banking and personal finance, our focus will be on examining how different demographics access and use financial services across various economies. The insights drawn will help highlight potential areas of financial inclusion and opportunities for improving financial services globally.

# **This is just a draft for what we could be writing to begin with**


## Data Cleaning and Manipulation

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Initially, the data couldn’t be read with UTF-8 due to special characters. Using ChatGPT we found that latin-1 supports these characters, allowing for proper file decoding.
# data = pd.read_csv('https://github.com/aaubs/ds-master/raw/main/data/assignments_datasets/FINDEX/WLD_2021_FINDEX_v03_M_csv.zip')

In [20]:
# Reading the data using the raw URL on github
data = pd.read_csv('https://github.com/aaubs/ds-master/raw/main/data/assignments_datasets/FINDEX/WLD_2021_FINDEX_v03_M_csv.zip', encoding='latin-1')

### Initial Data Structure Overview: Head, Info, Shape, Index, and Column Names

In [26]:
data.head()

,economy,economycode,regionwb,pop_adult,wpid_random,wgt,female,age,educ,inc_q,...,receive_transfers,receive_pension,receive_agriculture,pay_utilities,remittances,mobileowner,internetaccess,anydigpayment,merchantpay_dig,year
0,Afghanistan,AFG,South Asia,22647496.0,144274031,0.716416,2,43.0,2,4,...,4,4,4.0,1,5.0,1,2,1,0.0,2021
1,Afghanistan,AFG,South Asia,22647496.0,180724554,0.497408,2,55.0,1,3,...,4,4,2.0,4,5.0,1,2,0,0.0,2021
2,Afghanistan,AFG,South Asia,22647496.0,130686682,0.650431,1,15.0,1,2,...,4,4,4.0,4,3.0,2,2,0,0.0,2021
3,Afghanistan,AFG,South Asia,22647496.0,142646649,0.991862,2,23.0,1,4,...,4,4,2.0,4,5.0,1,2,0,0.0,2021
4,Afghanistan,AFG,South Asia,22647496.0,199055310,0.554940,1,46.0,1,1,...,4,4,4.0,4,5.0,2,2,0,0.0,2021


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143887 entries, 0 to 143886
Columns: 128 entries, economy to year
dtypes: float64(90), int64(35), object(3)
memory usage: 140.5+ MB


In [25]:
data.shape

(143887, 128)

In [23]:
data.index

RangeIndex(start=0, stop=143887, step=1)

In [24]:
data.columns

Index(['economy', 'economycode', 'regionwb', 'pop_adult', 'wpid_random', 'wgt',
       'female', 'age', 'educ', 'inc_q',
       ...
       'receive_transfers', 'receive_pension', 'receive_agriculture',
       'pay_utilities', 'remittances', 'mobileowner', 'internetaccess',
       'anydigpayment', 'merchantpay_dig', 'year'],
      dtype='object', length=128)

### Checking for missing values

In [40]:
# Do to the size of the Dataset we filter columns with missing values greater than 0. We look further into all the
# missing values with pd.set_option('display.max_rows', None) was used afterwards using pd.reset_option('display.max_rows')
pd.reset_option('display.max_rows')
missing_data = data.isnull().sum()
missing_data[missing_data > 0]

,0
regionwb,1000
age,467
emp_in,3502
urbanicity_f2f,68243
account_mob,61181
...,...
fin45_1,33106
fin45_1_China,140387
receive_agriculture,29606
remittances,29606


Its clear that there are significant missing values in the dataset. However, many of these missing values result from certain questions not being asked to all participants, which accounts for the bulk of the missing data. Additionally, we notice some missing information regarding the age, region, and employment status of participants. Since the missing values in these key categories (age, region, and workforce status) is quite small compared to the dataset size, it has been decided to remove these rows.

Additionally, columns with a high percentage of missing data (up to 90%) were removed, as they generally represent niche or specific follow-up questions. This simplification streamlines the dataset without affecting its overall quality, as our focus is on a broader analysis rather than specific topics like how domestic remittances were received.

In [46]:
# Removing the rows with missing values in 'regionwb', 'age', and 'emp_in'
data_cleaned = data.dropna(subset=['regionwb', 'age', 'emp_in'])

# Removing columns with more than 4000 missing values as this removes all "irrelevant" columns but not regionwb, age and emp_in
missing_threshold = 4000
cols_to_drop = data_cleaned.columns[data_cleaned.isnull().sum() > missing_threshold]

# Dropping the columns
data_cleaned.drop(columns=cols_to_drop, inplace=True)

# Cleaned dataset
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138973 entries, 0 to 143886
Data columns (total 42 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   economy            138973 non-null  object 
 1   economycode        138973 non-null  object 
 2   regionwb           138973 non-null  object 
 3   pop_adult          138973 non-null  float64
 4   wpid_random        138973 non-null  int64  
 5   wgt                138973 non-null  float64
 6   female             138973 non-null  int64  
 7   age                138973 non-null  float64
 8   educ               138973 non-null  int64  
 9   inc_q              138973 non-null  int64  
 10  emp_in             138973 non-null  float64
 11  account            138973 non-null  int64  
 12  account_fin        138973 non-null  int64  
 13  fin2               138973 non-null  int64  
 14  fin14_1            138973 non-null  int64  
 15  fin14a             138973 non-null  int64  
 16  fin14a1

<ipython-input-46-cd1297e0bc21>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=cols_to_drop, inplace=True)


**This leaves us with a dataset of 138973 rows and 42 columns with zero missing values**.

The remaining columns signify the following which we will be working with:

0. economy: The name of the country or economy.
1. economycode: ISO 3-digit code representing each economy.
2. regionwb: World Bank region classification (e.g., Sub-Saharan Africa, East
    Asia, etc.).
3.  pop_adult: The population of adults (aged 15+) in the economy.
4.  wpid_random: A unique identifier for each respondent in the dataset.
5.  wgt: Survey weight for each respondent, used to make the sample
    representative of the population.
6.  female: Gender of the respondent (1 if female, 2 if male).
7.  age: Age of the respondent.
8.  educ: Respondent’s education level form level 1 to 3
9.  inc_q: Income quintile of the respondent’s household.
10. emp_in: Employment status of the respondent.
11. account: Whether the respondent has an account at a financial institution
    or with a mobile money service provider.
12. account_fin: Whether the respondent has an account at a formal financial
    institution.
13. fin2: Has a debit card
14. fin14_1: Whether the respondent used mobile money.
15. fin14a: Made bill payments online using the Internet
16. fin14a1: Send money to a relative or friend online
    using the Internet
17. fin14b: Bought something online using the Internet
18. fin16: Saved for old age
19. fin17a: Saved using an account at a financial
    institution
20. fin20: Borrowed for medical purposes
21. fin22a: Borrowed from a financial institution
22. fin22b: Borrowed from family or friends
23. fin24: Main source of emergency funds in 30 days
24. fin30: Paid a utility bill
25. fin32: Received wage payments
26. fin37: Received a government transfer
27. fin38: Received a government pension
28. fin44a: Financially worried: old age
29. fin44b: Financially worried: medical cost
30. fin44c: Financially worried: bills
31. fin44d: Financially worried: education
32. saved: Saved money in the past 12 months.
33. borrowed: Borrowed money in the past 12 months.
34. receive_wages: Received a wage payment and method
35. receive_transfers: Received government transfers or aid payments and method
36. receive_pension: Received government pension payments. and method
37. pay_utilities: Paid utility bills and method
38. mobileowner: Whether the respondent owns a mobile phone.
39. internetaccess: Whether the respondent has access to the internet.
40. anydigpayment: Whether the respondent made any digital payment.
41. year: The year of the data collection